In [9]:
pip install vaderSentiment

   ---------------------------------------- 0.0/126.0 kB ? eta -:--:--
   --- ------------------------------------ 10.2/126.0 kB ? eta -:--:--
   ------ -------------------------------- 20.5/126.0 kB 217.9 kB/s eta 0:00:01
   ------------ -------------------------- 41.0/126.0 kB 326.8 kB/s eta 0:00:01
   -------------------------------------  122.9/126.0 kB 798.9 kB/s eta 0:00:01
   -------------------------------------- 126.0/126.0 kB 741.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
from fnmatch import fnmatch
import pandas as pd
import numpy as np
import nltk
import seaborn as sns
import spacy
import glob
import string
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import datetime
import re
from nltk.corpus import stopwords
from nltk import bigrams
from nltk.stem.wordnet import WordNetLemmatizer
from textblob import TextBlob
import itertools
import collections
import plotly.express as px
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

ModuleNotFoundError: No module named 'nltk'

In [2]:
import warnings
warnings.filterwarnings("ignore")

sns.set(font_scale=1.5)
sns.set_style("whitegrid")

In [55]:
data = pd.read_csv('data/Covid_Tweets_India.csv')

city_map = pd.read_csv('data/city_state_mapping_1.csv')

In [41]:
data.head()

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,...,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name.1,user_statuses_count,user_time_zone,user_urls,user_verified
0,"86.80749,23.77353",Fri Apr 10 04:52:47 +0000 2020,NaN,NaN,https://www.instagram.com/p/B-xE5k-hLep/?igshi...,2,1.248470e+18,NaN,NaN,NaN,...,59,352,0,"Maithon, Dhanbad",Suvam Paswan,SuvamPaswan,568,NaN,http://www.instagram.com/_mr.september_09,False
1,"91.7543716,26.1771297",Fri Apr 10 07:32:30 +0000 2020,No only no safe,NaN,http://Guwahati.city https://www.instagram.com...,1,1.248510e+18,NaN,NaN,NaN,...,9,218,0,Guwahati,Pallav Dwijen,PDwijen,182,NaN,NaN,False
2,"80.9381,26.8388",Fri Apr 10 06:31:47 +0000 2020,corona covid_19 staysafe jantacurfew support o...,NaN,https://www.instagram.com/p/B-ylbKdlgyE/?igshi...,1,1.248500e+18,NaN,NaN,NaN,...,16,197,0,"Lucknow, India",Oriental Consultants,lkoplacement,42,NaN,http://orientalconsultants.in/,False
3,"77.417,23.25",Fri Apr 10 08:01:39 +0000 2020,DilSeThankYou,NaN,https://www.instagram.com/p/B-yvtYghSpk/?igshi...,0,1.248520e+18,NaN,NaN,NaN,...,214,223,2,"Madhya Pradesh, India",Vinod Nagwanshi,MPJrSalesman,10837,NaN,https://twitter.com/mpjrsalesmen?s=08,False
4,"73.03333282,26.21345523",Fri Apr 10 07:38:29 +0000 2020,lockdown RajGardenPlants ROOKHRAJPAUDHSHALA co...,NaN,https://www.instagram.com/p/B-huTstnog5/?igshi...,0,1.248520e+18,NaN,NaN,NaN,...,2549,4557,21,"Raj Garden Plants, Jodhpur",Raj Garden Plants,rajsol29,18597,NaN,http://rajgardenplants.com,False


In [42]:
data.columns

Index(['coordinates', 'created_at', 'hashtags', 'media', 'urls',
       'favorite_count', 'id', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'lang', 'place',
       'possibly_sensitive', 'retweet_count', 'reweet_id',
       'retweet_screen_name', 'source', 'text', 'tweet_url', 'user_created_at',
       'user_screen_name', 'user_default_profile_image', 'user_description',
       'user_favourites_count', 'user_followers_count', 'user_friends_count',
       'user_listed_count', 'user_location', 'user_name', 'user_screen_name.1',
       'user_statuses_count', 'user_time_zone', 'user_urls', 'user_verified'],
      dtype='object')

In [43]:
text_data = data[['created_at','place','text']].copy()

In [44]:
text_data.head()

,created_at,place,text
0,Fri Apr 10 04:52:47 +0000 2020,"Maithon, India",:\nT H E - E A R T H - I S - H E A L I N G : ♻...
1,Fri Apr 10 07:32:30 +0000 2020,"Guwahati, India",##No excersise##only pose##no healthy food but...
2,Fri Apr 10 06:31:47 +0000 2020,"Lucknow, India","Remember the virus doesn't move,we move.\nWe s..."
3,Fri Apr 10 08:01:39 +0000 2020,"Bhopal, India","Thank you to all the doctors, nurses, para-med..."
4,Fri Apr 10 07:38:29 +0000 2020,"Jodhpur, India",Happy #lockdown Friends! Spending quality time...


In [45]:
text_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8695 entries, 0 to 8694
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   created_at  8695 non-null   object
 1   place       8695 non-null   object
 2   text        8695 non-null   object
dtypes: object(3)
memory usage: 203.9+ KB


In [57]:
#Adding Date Value
text_data['date'] = [datetime.datetime.strptime((x.split(" ").pop(2)+ "-"+x.split(" ").pop(1)+ "-"+x.split(" ").pop(5)),"%d-%b-%Y")  for x in text_data['created_at']]

text_data['city'] = [place.split(",")[0] for place in text_data['place']]

text_data = pd.merge(left=text_data,right=city_map[['City','State']],left_on='city',right_on='City',how='left')

text_data.head()

,created_at,place,text,date,city,City,State
0,Fri Apr 10 04:52:47 +0000 2020,"Maithon, India",:\nT H E - E A R T H - I S - H E A L I N G : ♻...,2020-04-10,Maithon,NaN,NaN
1,Fri Apr 10 07:32:30 +0000 2020,"Guwahati, India",##No excersise##only pose##no healthy food but...,2020-04-10,Guwahati,Guwahati,Assam
2,Fri Apr 10 06:31:47 +0000 2020,"Lucknow, India","Remember the virus doesn't move,we move.\nWe s...",2020-04-10,Lucknow,Lucknow,Uttar Pradesh
3,Fri Apr 10 08:01:39 +0000 2020,"Bhopal, India","Thank you to all the doctors, nurses, para-med...",2020-04-10,Bhopal,Bhopal,Madhya Pradesh
4,Fri Apr 10 07:38:29 +0000 2020,"Jodhpur, India",Happy #lockdown Friends! Spending quality time...,2020-04-10,Jodhpur,Jodhpur,Rajasthan


In [47]:
# Minimum and maximum date
print(f'minimum date of extraction is {text_data["date"].min().date()}')
print(f'maximum date of extraction is {text_data["date"].max().date()}')


minimum date of extraction is 2020-03-19
maximum date of extraction is 2020-07-01


In [63]:
text_data[pd.isnull(text_data['State'])].shape

(3704, 7)


### Analysis

#### Most-used #tags
#### Word cloud ,every word, bigrams
#### Sentiment analysis

In [16]:
def extract_hash_tags(s):
    return list(part for part in s.split() if part.startswith('#'))

In [18]:
# Fetching #tags used and count of #tags in tweets
tags = []
cnt=[]
for tweet in text_data['text']:
    tag = [i  for i in tweet.split() if i.startswith("#")]
    tags.append(tag)
    cnt.append(len(tag))
    

In [22]:
text_data['#tags'] = tags
text_data['#tags_cnt'] = cnt

In [23]:
text_data.head()

,created_at,place,text,#tags,#tags_cnt
0,Fri Apr 10 04:52:47 +0000 2020,"Maithon, India",:\nT H E - E A R T H - I S - H E A L I N G : ♻...,[],0
1,Fri Apr 10 07:32:30 +0000 2020,"Guwahati, India",##No excersise##only pose##no healthy food but...,[##No],1
2,Fri Apr 10 06:31:47 +0000 2020,"Lucknow, India","Remember the virus doesn't move,we move.\nWe s...","[#corona, #covid_19, #staysafe, #jantacurfew, ...",10
3,Fri Apr 10 08:01:39 +0000 2020,"Bhopal, India","Thank you to all the doctors, nurses, para-med...",[#DilSeThankYou],1
4,Fri Apr 10 07:38:29 +0000 2020,"Jodhpur, India",Happy #lockdown Friends! Spending quality time...,"[#lockdown, #RajGardenPlants, #ROOKHRAJPAUDHSH...",9


In [ ]:
tag_list=[tag_list.append(tag) for tag in text_data['#tags']]
# most_used_tags = set([tag_list.append(tag) for tag in text_data['#tags']])